DataJoint is a Python (or MATLAB) framework that allows manipulation of MySQL databases and tables within Python (or MATLAB).

Python (or MATLAB) classes to represent and link to MySQL tables. Manipulation of tables (create and drop tables, insert, delete and update entries) could be implemented by calling methods of table classes.

In [ ]:
import datajoint as dj

In [ ]:
# create schema, house of tables
schema = dj.schema('demo_nwb')

In [ ]:
# create a table by declaring a class
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id       : int 
    ---
    dob            : date            # date of birth
    sex            : enum('M', 'F', 'unknown')
    """

In [ ]:
# insert some mouse entries
data = [(0, '2019-08-25', 'M'),
        (1, '2019-10-21', 'F'),
        (2, '2020-01-01', 'unknown'),
        (3, '2019-02-03', 'M'),
        (4, '2019-08-23', 'F')
       ]

Mouse.insert(data)

In [ ]:
Mouse()

In [ ]:
# create another table representing experimental session
@schema
class Session(dj.Manual):
    definition = """
    -> Mouse
    session_date       : date
    ---
    experimenter       : varchar(32)
    setup              : int        
    """

In [ ]:
# insert some sessions
data = [
    {'mouse_id': 0, 'session_date': '2019-10-30', 'experimenter': 'Shan Shen', 'setup': 1},
    {'mouse_id': 0, 'session_date': '2019-11-02', 'experimenter': 'Shan Shen', 'setup': 1},
    {'mouse_id': 1, 'session_date': '2019-12-30', 'experimenter': 'Thinh Nguyen', 'setup': 2},
    {'mouse_id': 2, 'session_date': '2020-01-30', 'experimenter': 'Shan Shen', 'setup': 2},
]

In [ ]:
Session.insert(data)

In [ ]:
Session()

In [ ]:
Mouse & 'sex="M"'

In [ ]:
Mouse & Session

In [ ]:
(Mouse & Session).fetch()

In [ ]:
# delete entries
(Mouse & 'mouse_id=2').delete()

In [ ]:
# drop table
Mouse.drop()